In [2]:
from pytube import YouTube
from datetime import datetime
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import Playlist
import json
from urllib.parse import quote_plus
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

In [3]:
def get_transcript(video_id):
  transcript =  YouTubeTranscriptApi.get_transcript(video_id)
  full_transcript = " ".join([entry['text'] for entry in transcript])
  return full_transcript


def get_video_metadata(video_url):
    try:
        # Create a YouTube object
        yt = YouTube(video_url)
        
        # Extract metadata
        length = yt.length
        video_id = yt.video_id
        date = yt.publish_date.date()
        channel = yt.author
        title = yt.title
        topic = yt.keywords
        

   
        
        # Attempt to extract presenter and topic from title or description
        # This is a simple approach and may need refinement based on video naming conventions
        presenter = "Not specified"
   
           
      
        metadata={}
        metadata['Video ID'] = video_id
        metadata['Upload Date'] = date.isoformat()
        metadata['Channel'] = channel
        metadata['Presenter'] = presenter
        metadata['Title'] = title
        metadata['Length'] = length
        metadata['Topic'] = topic

        
        return metadata
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
def get_together(full_transcript,metadata):
  metadata['Transcript'] =  full_transcript
  return metadata

def write_json(metadata):
  json_string = json.dumps(metadata, indent=4)
  # Write JSON string to a file
  with open('video_metadata.json', 'w') as json_file:
    json_file.write(json_string)
  return 
    
def process_video(video_url):
    # Step 1: Get video metadata
    metadata = get_video_metadata(video_url)
    if metadata is None:
        return "Failed to retrieve metadata"
    
    # Step 2: Get transcript
    transcript = get_transcript(metadata['Video ID'])
    
    # Step 3: Combine metadata and transcript
    complete_data = get_together(transcript, metadata)
    
    # Step 4: Write to JSON file
    output_file = write_json(complete_data)
    
    return f"Pipeline completed."

In [4]:
'''
import os

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_transcript = " ".join([entry['text'] for entry in transcript])
        return full_transcript
    except Exception as e:
        print(f"Error getting transcript for video {video_id}: {str(e)}")
        return ""

def get_video_metadata(video_url):
    try:
        yt = YouTube(video_url)
        
        metadata = {
            'Video ID': yt.video_id,
            'Upload Date': yt.publish_date.date().isoformat() if yt.publish_date else "Unknown",
            'Channel': yt.author,
            'Presenter': "Not specified",
            'Title': yt.title,
            'Length': yt.length,
            'Topic': yt.keywords if yt.keywords else []
        }
        
        return metadata
    except Exception as e:
        print(f"Error getting metadata for video {video_url}: {str(e)}")
        return None

def get_together(full_transcript, metadata):
    metadata['Transcript'] = full_transcript
    return metadata

def write_json(metadata_list, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

def process_playlist(playlist_url, output_file):
    playlist = Playlist(playlist_url)
    all_video_metadata = []

    for video_url in playlist.video_urls:
        print(f"Processing video: {video_url}")
        
        # Step 1: Get video metadata
        metadata = get_video_metadata(video_url)
        if metadata is None:
            print(f"Failed to retrieve metadata for {video_url}")
            continue

        # Step 2: Get transcript
        transcript = get_transcript(metadata['Video ID'])

        # Step 3: Combine metadata and transcript
        complete_data = get_together(transcript, metadata)

        all_video_metadata.append(complete_data)

    # Step 4: Write all video metadata to JSON file
    write_json(all_video_metadata, output_file)

    return f"Pipeline completed. Processed {len(all_video_metadata)} videos. Output written to {output_file}"
'''

'\nimport os\n\ndef get_transcript(video_id):\n    try:\n        transcript = YouTubeTranscriptApi.get_transcript(video_id)\n        full_transcript = " ".join([entry[\'text\'] for entry in transcript])\n        return full_transcript\n    except Exception as e:\n        print(f"Error getting transcript for video {video_id}: {str(e)}")\n        return ""\n\ndef get_video_metadata(video_url):\n    try:\n        yt = YouTube(video_url)\n        \n        metadata = {\n            \'Video ID\': yt.video_id,\n            \'Upload Date\': yt.publish_date.date().isoformat() if yt.publish_date else "Unknown",\n            \'Channel\': yt.author,\n            \'Presenter\': "Not specified",\n            \'Title\': yt.title,\n            \'Length\': yt.length,\n            \'Topic\': yt.keywords if yt.keywords else []\n        }\n        \n        return metadata\n    except Exception as e:\n        print(f"Error getting metadata for video {video_url}: {str(e)}")\n        return None\n\ndef get

In [5]:

video_url = "https://www.youtube.com/watch?v=ybIVtRE-QHU"
result = process_video(video_url)
print(result)

Pipeline completed.


In [6]:
!pip install llama_index
!pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import StorageContext
from llama_index.core import Settings
from sqlalchemy import create_engine
import json
from llama_index.llms.ollama import Ollama
from transformers import pipeline

# Load the JSON file containing video data
with open('video_metadata.json', 'r') as file:
    video_data = json.load(file)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="Alibaba-NLP/gte-large-en-v1.5",trust_remote_code=True
)

llm = Ollama(model='gemma2:2b', request_timeout=900.0, base_url=f"http://10.0.0.180:11434")
Settings.llm = llm

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [8]:

db = {
    'db_host':      '10.0.0.180', 
    'db_port':      '5432', 
    'db_name':      'postgres', 
    'db_user':      'postgres', 
    'db_password':  'P0stGr3sP@ss', 
}

In [9]:
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document

In [10]:
table_name ="embeddings"
vector_store = PGVectorStore.from_params(
    database=db['db_name'],
    host=db['db_host'],
    password=quote_plus(db['db_password']),
    port=db['db_port'],
    user=db['db_user'],
    table_name= table_name,
    embed_dim=1024,  
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [11]:
from llama_index.core.schema import TextNode
yt_transcript_node = [
                            TextNode(text=video_data['Transcript'],
                                metadata={
                                    "Video_ID": video_data['Video ID'],
                                    "Upload Date": video_data['Upload Date'],
                                    "Title": video_data['Title'],
                                    "Topic": video_data.get('Topic', []),
                                    "Presenter": video_data['Presenter'],
                                    "Channel" : video_data['Channel'],
                                    "Length": video_data['Length']
                                },
                            )
                        ]
vector_index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store, 
        storage_context=storage_context,
    )


In [12]:
vector_index.insert_nodes(yt_transcript_node)
   

In [14]:
query_engine = vector_index.as_query_engine()


In [ ]:
# Example query
response = query_engine.query("What are the 2 main topics discussed in the videos?")
print(response)